# 目录
## 1. 模型导入
## 2. 签名函数
## 3. 签名函数---> 具体函数
## 4. 签名函数--->SavedModel

## 1. 模型导入

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn

from tensorflow import keras
import tensorflow as tf
import sys
import os
import time
import datetime

for module in [np, pd, mpl, sklearn, keras, tf]:
    print(module.__name__, module.__version__)
    
gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(gpus[0], True)

numpy 1.18.1
pandas 0.25.3
matplotlib 3.1.2
sklearn 0.22.1
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
tensorflow 2.1.0


## 2. 签名函数

In [3]:
# 签名函数
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name="x")])
def cube(x):
    return tf.pow(x, 3)

## 3. 签名函数---> 具体函数

In [4]:
# 签名函数---> 具体函数
cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None], tf.int32))
print(cube_func_int32(tf.constant([1, 2, 3, 4])))

tf.Tensor([ 1  8 27 64], shape=(4,), dtype=int32)


## 4. 签名函数--->SavedModel

In [6]:
# 签名函数--->SavedModel

to_export = tf.Module()
to_export.cube = cube
tf.saved_model.save(to_export, "./signature_to_savedModel")

INFO:tensorflow:Assets written to: ./signature_to_savedModel/assets


In [7]:
# 查看模型
!saved_model_cli show --dir ./signature_to_savedModel/ --all

2020-01-31 20:31:50.660675: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-01-31 20:31:50.662487: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['x'] tensor_info:
        dtype: DT_INT32
        shape: (-1)
        name: serving_default_x:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tens

In [10]:
# 加载模型  推理
model = tf.saved_model.load("./signature_to_savedModel/")

print(model.signatures.keys())

model.cube(tf.constant([1, 2, 3, 4]))

KeysView(_SignatureMap({'serving_default': <tensorflow.python.saved_model.load._WrapperFunction object at 0x7f83557d1198>}))


<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 1,  8, 27, 64], dtype=int32)>